# JULIA

In [1]:
# versioninfo()
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (arm64-apple-darwin22.4.0)
  CPU: 8 × Apple M1
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, apple-m1)
  Threads: 1 on 4 virtual cores
Environment:
  JULIA_NUM_THREADS = 


## Types

### Int64

In [2]:
v = 1_000_000

1000000

In [3]:
typeof(v)

Int64

### Float64

In [4]:
v = 10 / 4

2.5

In [5]:
typeof(v)

Float64

### Rational

In [6]:
v = 1//2 + 1//3

5//6

In [7]:
typeof(v)

Rational{Int64}

### Irrational

In [8]:
v = π # \pi

π = 3.1415926535897...

In [9]:
typeof(v)

Irrational{:π}

### Char

In [10]:
v = 'J'

'J': ASCII/Unicode U+004A (category Lu: Letter, uppercase)

In [11]:
typeof(v)

Char

In [12]:
# concat chars --> string!
'J' * 'u' * 'l' * 'i' * 'a'

"Julia"

### String

In [13]:
v = "Julia"

"Julia"

In [14]:
typeof(v)

String

In [15]:
# concat strings
"I" * " love " * "Julia" 

"I love Julia"

### Bool

In [16]:
v = true

true

In [17]:
typeof(v)

Bool

## Type conversion

### integer to float

In [18]:
convert(Float64, 10)

10.0

In [19]:
Float64(10)

10.0

### integer to string

In [20]:
string(10)

"10"

### integer to char

In [21]:
Char(74)

'J': ASCII/Unicode U+004A (category Lu: Letter, uppercase)

### float to integer

In [22]:
convert(Int64, 10.0)

10

In [23]:
Int64(10.0)

10

### float to string

In [24]:
string(3.14159)

"3.14159"

### string to integer

In [25]:
parse(Int64, "10")

10

### string to float

In [26]:
parse(Float64, "3.14159")

3.14159

### char to integer

In [27]:
Int64('J')

74

## `supertype` and `supertypes`

In [28]:
supertype(Int64)

Signed

In [29]:
supertype(Signed)

Integer

In [30]:
supertypes(Int64)

(Int64, Signed, Integer, Real, Number, Any)

In [31]:
Integer <: Number

true

In [32]:
Signed <: Int64

false

### `subtypes`

In [33]:
subtypes(Integer)

3-element Vector{Any}:
 Bool
 Signed
 Unsigned

In [34]:
subtypes(Signed)

6-element Vector{Any}:
 BigInt
 Int128
 Int16
 Int32
 Int64
 Int8

In [35]:
subtypes(Unsigned)

5-element Vector{Any}:
 UInt128
 UInt16
 UInt32
 UInt64
 UInt8

## Composite Types

### rectangle

In [36]:
struct rectangle
    length::Number
    width::Number
end

In [37]:
my_rectangle = rectangle(4.0, 2.0)

rectangle(4.0, 2.0)

In [38]:
my_rectangle.length, my_rectangle.width

(4.0, 2.0)

In [39]:
typeof(my_rectangle)

rectangle

### circle

In [40]:
struct circle
    radius::Number
end

In [41]:
my_circle = circle(10)

circle(10)

In [42]:
my_circle.radius

10

In [43]:
typeof(my_circle)

circle

### multiple dispatching!

In [44]:
# method rectange
function area(x::rectangle)
    return x.length * x.width
end

area (generic function with 1 method)

In [45]:
# method circle
function area(x::circle)
    return π * x.radius^2
end

area (generic function with 2 methods)

In [46]:
# area rectangle
area(my_rectangle)

8.0

In [47]:
# area circle
area(my_circle)

314.1592653589793

In [48]:
# methods
methods(area)

# 2 methods for generic function "area" from Main:
 [1] area(x::rectangle)
     @ ~/Documents/Projects/Julia/JuliaBasics/types.ipynb:2
 [2] area(x::circle)
     @ ~/Documents/Projects/Julia/JuliaBasics/types.ipynb:2

## `typemin` and `typemax`

https://docs.julialang.org/en/v1/manual/integers-and-floating-point-numbers

In [49]:
# Int8
typemin(Int8), typemax(Int8)

(-128, 127)

In [50]:
# Uint8
Int64(typemin(UInt8)), Int64(typemax(UInt8))

(0, 255)

In [51]:
# Int64
typemin(Int64), typemax(Int64)

(-9223372036854775808, 9223372036854775807)

In [52]:
# Int128
typemin(Int128), typemax(Int128)

(-170141183460469231731687303715884105728, 170141183460469231731687303715884105727)

## Overflow and Underflow

### Overflow

In [53]:
for i in Int8[5, 10, 15, 20]
    println(i, " -> ", i^2)
end


5 -> 25
10 -> 100
15 -> -31
20 -> -112


In [54]:
# function reproduce the overflow return values
function overflow(type, value)
    type_min, type_max = typemin(type), typemax(type)
    while value > type_max
        value -= length(type_min:type_max)
    end
    return value
end

overflow (generic function with 1 method)

In [55]:
# overflow 15^2
overflow(Int8, 15^2)

-31

In [56]:
# overflow 20^2
overflow(Int8, 20^2)

-112

### Underflow

In [57]:
# Int64
a = -128
b = 22
a - b

-150

In [58]:
# Underflow Int8
a = Int8(-128)
b = Int8(22)
a - b

106

### Solution I (widen datatype)

In [59]:
for i in Int8[5, 10, 15, 20]
    println(i, " -> ", widen(i)^2)
end

5 -> 25
10 -> 100
15 -> 225
20 -> 400


In [60]:
v = Int8(100)
w = widen(v)
typeof(w)

Int16

### Solution II (change input datatype)

In [61]:
for i in Int16[5, 10, 15, 20] # was Int8
    println(i, " -> ", i^2)
end

5 -> 25
10 -> 100
15 -> 225
20 -> 400


### Solution III (use checked functions)

In [62]:
for i in Int8[5, 10, 15, 20]
    println(i, " -> ", Base.Checked.checked_mul(i, i))
end

5 -> 25
10 -> 100


OverflowError: OverflowError: 15 * 15 overflowed for type Int8